In [1]:
#import dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
from config import yelp_key, census_key
from pprint import pprint
import requests

# Census API Key
c = Census(census_key, year=2013)


In [2]:
# Census API call
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)
census_pd.head()

,B01002_001E,B01003_001E,B17001_002E,B19013_001E,B19301_001E,NAME,zip code tabulation area
0,36.6,18450.0,10816.0,12041.0,7380.0,ZCTA5 00601,00601
1,38.6,41302.0,22409.0,15663.0,8463.0,ZCTA5 00602,00602
2,38.9,53683.0,26220.0,15485.0,9176.0,ZCTA5 00603,00603
3,37.3,6591.0,3721.0,15019.0,6383.0,ZCTA5 00606,00606
4,39.2,28963.0,14569.0,16707.0,7892.0,ZCTA5 00610,00610


In [3]:
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "NAME",
                                      "zip code tabulation area": "zip"})

In [4]:
census_pd.dtypes

Median Age           float64
Population           float64
Poverty Count        float64
Household Income     float64
Per Capita Income    float64
NAME                  object
zip                   object
dtype: object

In [5]:
census_pd['zip'] = census_pd['zip'].astype("int64")
census_pd.dtypes

Median Age           float64
Population           float64
Poverty Count        float64
Household Income     float64
Per Capita Income    float64
NAME                  object
zip                    int64
dtype: object

In [6]:
census_pd.columns

Index(['Median Age', 'Population', 'Poverty Count', 'Household Income',
       'Per Capita Income', 'NAME', 'zip'],
      dtype='object')

In [7]:
census_pd = census_pd.drop(columns=['NAME'])


In [8]:
census_pd.head()

,Median Age,Population,Poverty Count,Household Income,Per Capita Income,zip
0,36.6,18450.0,10816.0,12041.0,7380.0,601
1,38.6,41302.0,22409.0,15663.0,8463.0,602
2,38.9,53683.0,26220.0,15485.0,9176.0,603
3,37.3,6591.0,3721.0,15019.0,6383.0,606
4,39.2,28963.0,14569.0,16707.0,7892.0,610


In [9]:
hospitals_list = pd.read_csv("Hospitals_Ratings.csv")
hospitals_list.head()

,City,Facility Name,Provider Number,State,Address,Phone Number,zip,Latitude,Longitude,Yelp Rating
0,REDWOOD CITY,KAISER FOUNDATION HOSPITAL REDWOOD CITY LABORA...,05D0062466,CA,1100 VETERANS BLVD,6.502992e+09,94063,37.489331,-122.223976,3.0
1,SAN FRANCISCO,CHINESE HOSPITAL LABORATORY,05D0062669,CA,845 JACKSON ST,4.159822e+09,94133,37.795541,-122.409130,3.0
2,PALO ALTO,LUCILE PACKARD CHILDREN'S HOSPITAL AT,05D0062689,CA,725 WELCH RD,6.507240e+09,94304,37.435550,-122.174638,3.0
3,SAN MATEO,SAN MATEO MEDICAL CENTER LABORATORY,05D0062712,CA,222 W 39TH AVE,6.505732e+09,94403,37.531225,-122.299138,2.5
4,ANTIOCH,SUTTER DELTA MEDICAL CENTER,05D0062749,CA,3901 LONE TREE WAY,9.257793e+09,94509,37.982535,-121.802908,2.0


In [10]:
hospitals_list.dtypes

City                object
Facility Name       object
Provider Number     object
State               object
Address             object
Phone Number       float64
zip                  int64
Latitude           float64
Longitude          float64
Yelp Rating        float64
dtype: object

In [11]:
updated_list = pd.merge(hospitals_list, census_pd, how="left", on="zip")
updated_list.head()

,City,Facility Name,Provider Number,State,Address,Phone Number,zip,Latitude,Longitude,Yelp Rating,Median Age,Population,Poverty Count,Household Income,Per Capita Income
0,REDWOOD CITY,KAISER FOUNDATION HOSPITAL REDWOOD CITY LABORA...,05D0062466,CA,1100 VETERANS BLVD,6.502992e+09,94063,37.489331,-122.223976,3.0,31.0,31497.0,6154.0,52818.0,22108.0
1,SAN FRANCISCO,CHINESE HOSPITAL LABORATORY,05D0062669,CA,845 JACKSON ST,4.159822e+09,94133,37.795541,-122.409130,3.0,40.4,27970.0,5189.0,50485.0,45147.0
2,PALO ALTO,LUCILE PACKARD CHILDREN'S HOSPITAL AT,05D0062689,CA,725 WELCH RD,6.507240e+09,94304,37.435550,-122.174638,3.0,37.6,3797.0,163.0,103971.0,81131.0
3,SAN MATEO,SAN MATEO MEDICAL CENTER LABORATORY,05D0062712,CA,222 W 39TH AVE,6.505732e+09,94403,37.531225,-122.299138,2.5,40.3,39720.0,1815.0,90764.0,46432.0
4,ANTIOCH,SUTTER DELTA MEDICAL CENTER,05D0062749,CA,3901 LONE TREE WAY,9.257793e+09,94509,37.982535,-121.802908,2.0,34.0,62651.0,11190.0,53953.0,22690.0


In [52]:
grouped_data = updated_list.groupby(['City'])

grouped_data.count().head(10)



,Facility Name,Provider Number,State,Address,Phone Number,zip,Latitude,Longitude,Yelp Rating,Median Age,Population,Poverty Count,Household Income,Per Capita Income
City,,,,,,,,,,,,,,
ALAMEDA,1,1,1,1,1,1,1,1,1,1,1,1,1,1
ANTIOCH,7,7,7,7,7,7,7,7,7,7,7,7,7,7
BERKELEY,8,8,8,8,8,8,8,8,8,8,8,8,8,8
BURLINGAME,13,13,13,13,13,13,13,13,13,13,13,13,13,13
CAMPBELL,4,4,4,4,4,4,4,4,4,4,4,4,4,4
CONCORD,12,12,12,12,12,12,12,12,12,9,9,9,9,9
DALY CITY,3,3,3,3,3,3,3,3,3,3,3,3,3,3
DUBLIN,1,1,1,1,1,1,1,1,1,1,1,1,1,1
EMERYVILLE,2,2,2,2,2,2,2,2,2,2,2,2,2,2


In [53]:
hospital_counts = grouped_data["zip"].count()
#hospital_counts.head()

In [54]:
#summary_table = pd.DataFrame({"Number of facilities": hospital_counts,})
#summary_table.head()

In [55]:
population_counts = grouped_data["Population"].sum()
#population_counts.head()

In [56]:
median_age = grouped_data["Median Age"].mean()
#median_age.head()

In [57]:
income = grouped_data ["Household Income"].mean()
#income.head()

In [58]:
poverty_count = grouped_data["Poverty Count"].sum()
#poverty_count.head()

In [59]:
per_capita_income = grouped_data["Per Capita Income"].mean()
#per_capita_income.head()

In [61]:
pd.options.display.float_format = '{:.2f}'.format
summary_table = pd.DataFrame({"Number of facilities": hospital_counts,
                             "Total Population":population_counts,
                             "Median Age":median_age,
                             "Average Income": income,
                             "Poverty Count":poverty_count,
                             "Per Capita Income":per_capita_income})
summary_table.head()

,Number of facilities,Total Population,Median Age,Average Income,Poverty Count,Per Capita Income
City,,,,,,
ALAMEDA,1,61639.00,39.40,69076.00,7125.00,38185.00
ANTIOCH,7,396655.00,33.86,63659.86,64258.00,24125.14
BERKELEY,8,98406.00,36.45,67640.38,13042.00,45328.88
BURLINGAME,13,539240.00,41.90,109775.00,32305.00,74474.00
CAMPBELL,4,180748.00,37.30,87204.00,13368.00,44023.00
